In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import pdb
db = pdb.set_trace
# Option 1.1 channel_state: number of users (easy)
# Option 1.2 channel_state: interference power (hard)
# Option 2.1 central learning and distributed executation (easy)
# Option 2.2 distributed learing and distributed executation (hard)
# Option 3.1 each agent has the same number of tasks (easy)
# Option 3.2 each agent has different number of tasks (middle)
# Option 3.3 each agent randomly generates tasks over time (hard)
# Option 4.1 env give back reward every N step (waste bandwidth but easy)
# Option 4.2 env only give back reward at last (save bandwidth but hard)
# Option 5: adjust reward of communicate/tear/observe
# Option 6.1: fully aware the env (do not need observe and easy)
# Option 6.2: part aware the env (need to decide wether to observe and hard)
# Option 7.1: action_args_pair = f(status) (easy but not scalable)
#             number of output: num(action) x num(args)
# Option 7.2: action, args = f(status) (hard but scalable)
#             number of output: num(action) + num(args)
# Option 8.1: use a step to do communicate schedule
# Option 8.2: do not use a step to do communicate schedule

In [ ]:
class Net(nn.Module):
    def __init__(self, in_features, num_actions):
        """deep Q-learning network for testing algorithm
           in_features: number of features of input.
           num_actions: number of action-value, each corresponde to a action 
        """
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_actions)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

In [ ]:
class LinearSchedule(object):
    def __init__(self, schedule_timesteps, final_p, initial_p=1.0):
        """Linear interpolation between initial_p and final_p over
        schedule_timesteps. After this many timesteps pass final_p is
        returned.
        Parameters
        ----------
        schedule_timesteps: int
            Number of timesteps for which to linearly anneal initial_p
            to final_p
        initial_p: float
            initial output value
        final_p: float
            final output value
        """
        self.schedule_timesteps = schedule_timesteps
        self.final_p            = final_p
        self.initial_p          = initial_p
        self.t = 0

    def value(self):
        """See Schedule.value"""
        self.t += 1
        fraction  = min(float(self.t) / self.schedule_timesteps, 1.0)
        return self.initial_p + fraction * (self.final_p - self.initial_p)

In [ ]:
class DQN(object):
    def __init__(self):
        self.eval_net = Net(N_STATES, N_ACTIONS).cuda()
        self.target_net = Net(N_STATES, N_ACTIONS).cuda()
        
        self.schedular = LinearSchedule(EPSILON_STEPS, EPSILON)
        
        self.learn_step_counter = 0                                     
        self.memory_counter = 0                                        
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2)) 
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0).cuda()
        # input only one sample
        if np.random.uniform() > self.schedular.value():   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].cpu().numpy()[0]
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]).cuda()
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)).cuda()
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]).cuda()
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]).cuda()

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        # detach from graph, don't backpropagate
        q_next = self.target_net(b_s_).detach()     
        q_target = b_r + GAMMA * q_next.max(1)[0]   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
class Environment(object):
    """Currently no propogation effect is considered
       The central station:
       (1) only provide signaling channel and do not schedule at all
       (2) monitor malicious users
    """

    def __init__(self, 
                 channel_num,
                 status_len,
                 agent_num, 
                 task_num, 
                 max_steps,
                 reward_interval):
        """channel_num: number of RF channels in the environment
           status_len: use how many channel_state to construct env status
           agent_num: number of agents to operate in this env
           task_num: the number of tasks in each agent
           max_steps: the number of steps in one epoch (ie. 50)
                      when max_steps is reached game over and give reward
           reward_interval: env give reward every reward_interval steps
           
           self.channel_state: log the channel user in this step
           self.history: store channel occupancy of the past
           self.step_index: log how many steps has gone
           self.agent_list: log the agent in this environment
           self.success_list: count success tasks of each agent
           self.conflict_list: count conflict tasks of each agent
           self.tmp_success_list: count success tasks over reward_interval
           self.tmp_conflict_list: count conflict tasks over reward_interval
        """
        self.channel_num = channel_num
        self.channel_state = [[] for i in range(self.channel_num)]
        
        self.status_len = status_len
        self.history = [[[] for i in range(self.channel_num)] 
                        for j in range(self.status_len)]
        
        self.agent_num = agent_num
        self.task_num = task_num
        
        self.max_steps = max_steps
        self.step_index = 0
        
        self.reward_interval = reward_interval
        
        self.agent_list = []
        self.success_list = []
        self.conflict_list = []
        
        self.tmp_success_list = []
        self.tmp_conflict_list = []
        
        self.dqn = DQN()
        
    # start to run the env with agents    
    def game_on(self): 
        for ep in range(EPOCH_NUM):
            self.reset()
            for i in range(self.agent_num):
                self.join(Agent(self))
            while True:
                done = self.step()
                if self.dqn.memory_counter > MEMORY_CAPACITY:
                    self.dqn.learn()
                if done:
                    if ep % PRINT_FREQ == 0: 
                        print("Epoch: {0:4d}".format(ep), end='  ')
                        self.report(verbose=False)
                    break
                
    # reset the env            
    def reset(self):
        self.channel_state = [[] for i in range(self.channel_num)]
        self.history = [[[] for i in range(self.channel_num)] 
                        for j in range(self.status_len)]
        self.step_index = 0
        self.agent_list = []
        self.success_list = []
        self.conflict_list = []
        self.tmp_success_list = []
        self.tmp_conflict_list = []
    
    # add agents into the env
    def join(self, agent):
        self.agent_list.append(agent)
        self.success_list.append(0)
        self.conflict_list.append(0)
        
        self.tmp_success_list.append(0)
        self.tmp_conflict_list.append(0)
    
    # run one step
    def step(self):
        """enter to next time step and initialize channel state
           call step method of all agents
           save channel state
        """
        self.step_index += 1  
        self.channel_state = [[] for i in range(self.channel_num)]
        for agent in self.agent_list:
            agent.step()
        #evaluate the transmission of each agent
        self.ber()
        self.history.append(self.channel_state)
        done = 0 if self.step_index < self.max_steps else 1
        return done

    #evaluate ber
    def ber(self):
        """count the success task and conflict task
        """
        for state in self.channel_state:
            if len(state) == 1:
                agent = state[0]
                index = self.agent_list.index(agent)
                self.success_list[index] += 1
                self.tmp_success_list[index] += 1
            if len(state) > 1:
                for agent in state:
                    index = self.agent_list.index(agent)
                    self.conflict_list[index] += 1
                    self.tmp_conflict_list[index] += 1
    
    # return all agents
    #spread communication message to all agents
    def broadcast(self):
        return self.agent_list

    def propagation(self, channel_index, agent):
        """propagate the signal of a certain agent"""
        self.channel_state[channel_index].append(agent)

    def query(self, channel_index):
        """return agents which occupied the channel
           which can be implemented by signal classification in experiment
        """
        return self.history[-1][channel_index]

    #Option 6.2
    def sense(self):
        """return the number of channel users
           which can be implemented by spectrum sensing in experiment
        """
        return [len(l) for l in self.history[-1]]
    
    #Option 6.1
    def status(self, flat):
        """return the status of the env"""
        st = [[len(l) for l in self.history[-(j+1)]] 
                      for j in range(self.status_len)]
        if flat:
            stf = []
            for l in st:
                stf += l
            return stf
        else:
            return st
    
    # Option 4.2
    def get_reward(self, agent):
        # The instructor is the environment (receiver),
        # which can evaluate how good the agent is doing by checksum.
        # The reward message is passed over signaling channel.
        # Since the frequent receiver-to-agent interaction wastes bandwidth,
        # the agent can only get back reward after its tasks are all finished
        # or the maximum time step is reached.
        
        if self.step_index < self.max_steps-1:
            return 0
        if self.step_index == self.max_steps-1:
            index = self.agent_list.index(agent)
            return self.success_list[index]+self.conflict_list[index]
        
    # Option 4.1
    def tmp_get_reward(self, agent):

        if self.step_index % self.max_steps == 0 \
           or self.step_index == self.max_steps:
            
            index = self.agent_list.index(agent)
            reward = (self.tmp_success_list[index]
                      +self.tmp_conflict_list[index])
            self.tmp_success_list[index] = 0
            self.tmp_conflict_list[index] = 0
            return reward
        else:
            return 0
    
    def report(self, verbose):
        """report status of all agent"""
        if verbose:
            for index,agent in enumerate(self.agent_list):
                print("success:{0:<4d}, "
                      "conflict:{1:<4d}, "
                      "remain:{2:<4d}, "
                      "channel:{3:<3d}, "
                      "reward: {4:<4.4f}".format(
                          self.success_list[index],
                          self.conflict_list[index],
                          agent.report()[0],
                          sum(agent.report()[1]),
                          agent.report()[2],
                      ))
            print("success rate: %f"
                  % (sum(self.success_list)/(self.task_num*self.agent_num)))
            #visulize spectrum use and conflict
            spec_his = np.array([[len(x) for x in y] for y in self.history])
            plt.matshow(spec_his.T)
            plt.colorbar()
        else:
            r = []
            for index,agent in enumerate(self.agent_list):
                r.append(agent.report()[2])
                print("{0:4.2f}".format(r[-1]), end=", ")
            r = np.array(r)
            print("mean:{0:4.2f}, std:{1:4.2f}".format(np.mean(r),np.std(r)))
            

In [ ]:
class Agent(object):
    """at the beginning, random trasmit
       then mainly use coordinate to gain channel
    """

    def __init__(self, env):
        """env: the environment to operate in
           self.channels: established channels (coordinated with receiver)
                          0-not established, 1-established
           self.part_state: observed channel states
        """
        self.env = env
        self.task_num = self.env.task_num
        self.channels = [0 for i in range(self.env.channel_num)]
        self.part_state = []
        
        self.operation_dict = {1:self.rest,
                               2:self.observe,
                               3:self.transmit,
                               4:self.tear,
                               5:self.establish,
                               6:self.communicate}
        
        self.dqn = self.env.dqn
        
        # action is (operation, channel_index)
        action_id = 0
        self.action_dict = {}
        for _, operation in self.operation_dict.items():
            args_num = operation.__code__.co_argcount
            if args_num == 1:
                self.action_dict[action_id] = (operation,0)
                action_id += 1
            else:
                for i in range(self.env.channel_num):
                    self.action_dict[action_id] = (operation,i)
                    action_id += 1
        
        self.status_curve = []
        self.action_curve = []
        self.reward_curve = []
    
    # Option 8.2
    def step(self):
        """interact with environment and agents within in one time step
           using Q-learning to decide whether to tear channel when job finished
        """
        self.reward_curve.append(0)
        if self.task_num > 0:
            # job has not finished at last step, punish for the delay
            # since we want to finish tasks as soon as possible
            self.reward_curve[-1] -= 0.5
        
#         # TODO: Q learning to choose operation (function and args)
#         # func: rest,transmit,establish,tear,observe,communicate
#         # args: channel_index
#         # channel_index: ie. 0~10
#         # status: task_num,channels,part_state
#         # find f: function, channel_index, target_agent_id = f(status)
        
        s = [self.task_num] + self.channels + self.env.status(flat=True)
        self.status_curve.append(s)
        a = self.dqn.choose_action(s)
#         a = random.randint(0, 32)
        operation, channel_index = self.action_dict[a]
        operation(channel_index)
        self.action_curve.append(a)
        
#         operation_index = random.randint(1,6)
#         channel_index = random.randint(0,self.env.channel_num-1)
#         self.operation_dict[operation_index](channel_index)

        self.reward_curve[-1] += self.env.tmp_get_reward(self)
        if len(self.status_curve)>2:
            self.dqn.store_transition(self.status_curve[-2],
                                      self.action_curve[-2],
                                      self.reward_curve[-2],
                                      self.status_curve[-1],
                                     )

    def rest(self, *args):
        """reset in this time step and do nothing
        """
        return
    
    def transmit(self, *args):
        """transmit using all established channels
           TODO: choose channel based on certain policy
        """
        for index, ocp in enumerate(self.channels):
            if self.task_num == 0: return
            if ocp:
                self.env.propagation(index, self)
                self.task_num -= 1

    def establish(self, channel_index, *args):
        """establish a new channel by coordinate with receiver
           channel_index: the index of channel to occupy
        """
        if self.channels[channel_index] == 0:
            self.channels[channel_index] = 1
            # Since expand operation need signaling bandwidth to coordinate
            self.reward_curve[-1] -= 1

    def tear(self, channel_index, *args):
        """tear down a channel by coordinate with receiver
           channel_index: the index of channel to release
        """
        if self.channels[channel_index] == 1:
            self.channels[channel_index] = 0
            # Shrink operation do not need signaling bandwidth to coordinate (0)
            # use the current data channel to trasmit tear down signal
            # Shrink operation ficilitate collabaration(+1)
            # decrease the possibility to conflict
            self.reward_curve[-1] += 1

    def observe(self, *args):
        """observe the channel usages
           TODO: may directly return the channel with highest availability
           TODO: in this stage, we let Q-learning find the candidate channel
        """
        state = copy.deepcopy(self.env.sense())
        self.part_state.append(state.append(self.env.step_index)) 
        # Since observe operation need energy to detect occupancy
        self.reward_curve[-1] -= 0.2
        # TODO find the channel with highest availability (least occupied)
    
    # abandon p2p communicate
    # reason 1: p2p communicate -> feedback are not stable
    # ie. A:0.1, B:0.3, C:0.2, multiple communicate may overwrite each other
    # reason 2: identify result is not reliable
    # since you do not exactly know whether the agent is still using the channel
    # so use broadcast communicate
    def communicate(self, channel_index, *args):
        """communicate with other agents over signaling channel
           the priority and desired channel is broadcast to all other agents           target: the target agent to communicate with
           protocal: (1) exchange priority score to collabration
                     (2) protect agents using different protocals
           since can not collaborate with them means can not use the channel
           and you will be interferenced when you do
        """
        # Since communicate need signaling bandwidth to coordinate (-1)
        self.reward_curve[-1] -= 1
        score = self.priority()
        for agent in self.env.broadcast():
            if agent.insist(channel_index, score): break
        else:
            self.establish(channel_index)
            
    # TODO: log communicate result and use Q-learning decide what to do next
    # Infeasible： since different agents are difficult to reach consensus

    def insist(self, channel_index, score):
        """schedule tear down operation if score is higher
           return the priority score of this agent
        """
        if self.channels[channel_index] == 0:
            return False
        if score > self.priority():
            self.tear(channel_index)
            return False
        else:
            return True
        
    def priority(self):
        """calculate priority score, or loss of the agent
           possible metric is to combine tasks num and self.reward_curve
           need to add constrict over score to avoid malicious deception
           通过（1）设备入网审查；（2）监测加入时间和后续发送数，保证score真实性
        """
        return self.task_num - sum(self.channels) - sum(self.reward_curve)
    
    def report(self):
        return self.task_num, self.channels, sum(self.reward_curve)

In [ ]:
# Hyper Parameters
CHANNEL_NUM = 10
STATUS_LEN = 5
AGENT_NUM = 5
TASK_NUM = 100
MAX_STEPS = 50
REWARD_INTERVAL = 10
N_ACTIONS = 3 + 3*CHANNEL_NUM
N_STATES = CHANNEL_NUM*STATUS_LEN + CHANNEL_NUM + 1

EPOCH_NUM = 1000
PRINT_FREQ = 50

BATCH_SIZE = 256
LR = 0.01                   # learning rate
EPSILON = 0.1              # greedy policy
EPSILON_STEPS = int(0.8*AGENT_NUM*MAX_STEPS*EPOCH_NUM)
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = int(0.1*AGENT_NUM*MAX_STEPS*EPOCH_NUM)

In [ ]:
e = Environment(channel_num=CHANNEL_NUM,
                 status_len=STATUS_LEN,
                 agent_num=AGENT_NUM, 
                 task_num=TASK_NUM, 
                 max_steps=MAX_STEPS,
                 reward_interval=REWARD_INTERVAL)
e.game_on()